## Projeto 02 - Extra - Como analisar os dados de Rentabilidade

### Desafio:

Construir um código que faça um ranking dos melhores fundos em três períodos de rentabilidade diferentes.
Iremos selecionar os 10 melhores posicionados no quesito rentabilidade no curto e no longo prazo.

### Escolher o navegador

In [1]:
!pip install webdriver-manager

In [2]:
!pip install selenium


In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

In [27]:
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

driver.get('https://www.etf.com/etfanalytics/etf-finder')

### Achar todos os elementos necessários dentro do HTML do site Expandindo a tabela para 100 itens.



In [28]:
time.sleep(5)

botao_100 = driver.find_element('xpath',
                               '/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[1]/div/div[4]/button/label/span')

driver.execute_script('arguments[0].click();', botao_100)

#outra opção de código para clicar caso o .click não funcione
#driver.execute_script('arguments[0].click();', botao_100)

### Achar todos os elementos necessários dentro do HTML  do site - Pegando o número de páginas da tabela.

In [29]:
numero_paginas = driver.find_element('xpath', '//*[@id="totalPages"]')
numero_paginas = numero_paginas.text.replace('of ','')
numero_paginas = int(numero_paginas)

numero_paginas

32

### Lendo a tabela de dados - Lendo a tabela de dados básicos.

In [30]:
elemento = driver.find_element('xpath','//*[@id="finderTable"]')

  
html_tabela = elemento.get_attribute('outerHTML')
    
tabela = pd.read_html(str(html_tabela))[0]

tabela

,Ticker,Name,Segment,Issuer,Expense Ratio,AUM
0,SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$374.42B
1,IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$306.66B
2,VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$287.12B
3,VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$282.79B
4,QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$172.68B
...,...,...,...,...,...,...
95,VXF,Vanguard Extended Market ETF,Equity: U.S. - Extended Market,Vanguard,0.06%,$14.04B
96,BIV,Vanguard Intermediate-Term Bond ETF,"Fixed Income: U.S. - Broad Market, Broad-based...",Vanguard,0.04%,$14.02B
97,XLI,Industrial Select Sector SPDR Fund,Equity: U.S. Industrials,State Street Global Advisors,0.10%,$13.89B
98,HYG,iShares iBoxx USD High Yield Corporate Bond ETF,"Fixed Income: U.S. - Corporate, Broad-based Hi...",Blackrock,0.48%,$13.73B


In [32]:
lista_tabela_por_pagina =[]

elemento = driver.find_element('xpath','//*[@id="finderTable"]')

for pagina in range(1, numero_paginas +1):
  
    html_tabela = elemento.get_attribute('outerHTML')
    
    tabela = pd.read_html(str(html_tabela))[0]
    
    lista_tabela_por_pagina.append(tabela)
   
    botao_avancar_pagina = driver.find_element('xpath','//*[@id="nextPage"]')

    driver.execute_script("arguments[0].click();", botao_avancar_pagina)
    
tabela_cadastro_etfs = pd.concat(lista_tabela_por_pagina)

tabela_cadastro_etfs

,Ticker,Name,Segment,Issuer,Expense Ratio,AUM
0,SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$374.42B
1,IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$306.66B
2,VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$287.12B
3,VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$282.79B
4,QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$172.68B
...,...,...,...,...,...,...
20,CVLC,Calvert US Large-Cap Core Responsible Index ETF,Equity: U.S. - Large Cap,Morgan Stanley,0.15%,--
21,CVMC,Calvert US Mid-Cap Core Responsible Index ETF,Equity: U.S. - Mid Cap,Morgan Stanley,0.15%,--
22,CVIE,Calvert International Responsible Index ETF,Equity: Developed Markets Ex-U.S. - Large Cap,Morgan Stanley,0.18%,--
23,CDEI,"Calvert US Large-Cap Diversity, Equity and Inc...",Equity: U.S. - Large Cap,Morgan Stanley,0.14%,--


### Ler a tabela de dados - Preenchendo um campo no site para voltar as páginas

In [33]:
formulario_de_voltar_pagina = driver.find_element('xpath', ' //*[@id="goToPage"]')

formulario_de_voltar_pagina.clear()
formulario_de_voltar_pagina.send_keys('1')
formulario_de_voltar_pagina.send_keys(u'\ue007')

### Ler a tabela de dados - Lendo a tabela de dados de rentabilidade

In [34]:
botao_mudar_pra_performance = driver.find_element('xpath','/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/ul/li[2]/span')

driver.execute_script('arguments[0].click();', botao_mudar_pra_performance)


# a partir daqui é tudo igual 

lista_tabela_por_pagina =[]

elemento = driver.find_element('xpath','//*[@id="finderTable"]')

for pagina in range(1, numero_paginas +1):
  
    html_tabela = elemento.get_attribute('outerHTML')
    
    tabela = pd.read_html(str(html_tabela))[0]
    
    lista_tabela_por_pagina.append(tabela)
   
    botao_avancar_pagina = driver.find_element('xpath','//*[@id="nextPage"]')

    driver.execute_script('arguments[0].click();', botao_avancar_pagina)
    
tabela_rentabilidade_etfs = pd.concat(lista_tabela_por_pagina)



In [11]:
tabela_rentabilidade_etfs

,Ticker,Name,1 Month,3 Month,YTD,1 Year,3 Years,5 Years,10 Years,As Of Date
0,SPY,SPDR S&P 500 ETF Trust,1.50%,6.90%,7.27%,-9.01%,19.91%,11.05%,12.19%,04/04/23
1,IVV,iShares Core S&P 500 ETF,1.49%,6.91%,7.27%,-9.02%,19.97%,11.06%,12.24%,04/04/23
2,VOO,Vanguard S&P 500 ETF,1.53%,6.94%,7.29%,-9.02%,19.98%,11.07%,12.24%,04/04/23
3,VTI,Vanguard Total Stock Market ETF,0.25%,6.24%,6.75%,-10.30%,19.93%,10.25%,11.74%,04/04/23
4,QQQ,Invesco QQQ Trust,6.63%,20.25%,20.01%,-12.94%,21.03%,15.65%,17.73%,04/04/23
...,...,...,...,...,...,...,...,...,...,...
0,CVLC,Calvert US Large-Cap Core Responsible Index ETF,0.41%,--,--,--,--,--,--,04/04/23
1,CVMC,Calvert US Mid-Cap Core Responsible Index ETF,-5.55%,--,--,--,--,--,--,04/04/23
2,CVIE,Calvert International Responsible Index ETF,1.81%,--,--,--,--,--,--,04/04/23
3,CDEI,"Calvert US Large-Cap Diversity, Equity and Inc...",2.18%,--,--,--,--,--,--,04/04/23


In [35]:
driver.quit()

## Construindo a Tabela Final

In [36]:
tabela_rentabilidade_etfs = tabela_rentabilidade_etfs.set_index('Ticker')
tabela_cadastro_etfs = tabela_cadastro_etfs.set_index('Ticker')

In [38]:
tabela_rentabilidade_etfs = tabela_rentabilidade_etfs[['1 Year','3 Years','5 Years']]

In [39]:
tabela_rentabilidade_etfs

,1 Year,3 Years,5 Years
Ticker,,,
SPY,-9.01%,19.91%,11.05%
IVV,-9.02%,19.97%,11.06%
VOO,-9.02%,19.98%,11.07%
VTI,-10.30%,19.93%,10.25%
QQQ,-12.94%,21.03%,15.65%
...,...,...,...
CVLC,--,--,--
CVMC,--,--,--
CVIE,--,--,--


## Escolher quais perídos de rentabilidade iremos analisar

## Retirar o dados faltantes. Fundos que não possuem períodos necessários, ficarão de fora.

In [40]:
tabela_rentabilidade_etfs = tabela_rentabilidade_etfs.replace("--", pd.NA)


In [41]:
tabela_rentabilidade_etfs = tabela_rentabilidade_etfs.dropna()

In [42]:
tabela_rentabilidade_etfs

,1 Year,3 Years,5 Years
Ticker,,,
SPY,-9.01%,19.91%,11.05%
IVV,-9.02%,19.97%,11.06%
VOO,-9.02%,19.98%,11.07%
VTI,-10.30%,19.93%,10.25%
QQQ,-12.94%,21.03%,15.65%
...,...,...,...
LD,-9.42%,10.26%,-2.12%
ERUS,0%,-33.19%,-22.74%
LTL,-26.30%,12.48%,1.36%


In [43]:
#Os dados vem em formato de texto e temos que transformá-los em decimal

for coluna in tabela_rentabilidade_etfs.columns:
    tabela_rentabilidade_etfs[coluna] = tabela_rentabilidade_etfs[coluna].str.rstrip("%").astype(float)/100


In [44]:
tabela_rentabilidade_etfs

,1 Year,3 Years,5 Years
Ticker,,,
SPY,-0.0901,0.1991,0.1105
IVV,-0.0902,0.1997,0.1106
VOO,-0.0902,0.1998,0.1107
VTI,-0.1030,0.1993,0.1025
QQQ,-0.1294,0.2103,0.1565
...,...,...,...
LD,-0.0942,0.1026,-0.0212
ERUS,0.0000,-0.3319,-0.2274
LTL,-0.2630,0.1248,0.0136


## Juntar as tabelas, igual na aula 2

In [45]:
base_final = tabela_cadastro_etfs.join(tabela_rentabilidade_etfs, how = "inner")

base_final

,Name,Segment,Issuer,Expense Ratio,AUM,1 Year,3 Years,5 Years
Ticker,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$374.42B,-0.0901,0.1991,0.1105
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$306.66B,-0.0902,0.1997,0.1106
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$287.12B,-0.0902,0.1998,0.1107
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$282.79B,-0.1030,0.1993,0.1025
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$172.68B,-0.1294,0.2103,0.1565
...,...,...,...,...,...,...,...,...
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$1.24M,-0.0942,0.1026,-0.0212
ERUS,iShares MSCI Russia ETF,Equity: Russia - Total Market,Blackrock,0.08%,$961.13K,0.0000,-0.3319,-0.2274
LTL,ProShares Ultra Communication Services,Leveraged Equity: U.S. Communication Services,ProShares,0.95%,$692.87K,-0.2630,0.1248,0.0136


## Filtrar os ETFs alavancados


In [47]:
base_final = base_final[~base_final ['Segment'].str.contains ('Leveraged')]

base_final

,Name,Segment,Issuer,Expense Ratio,AUM,1 Year,3 Years,5 Years
Ticker,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$374.42B,-0.0901,0.1991,0.1105
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$306.66B,-0.0902,0.1997,0.1106
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$287.12B,-0.0902,0.1998,0.1107
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$282.79B,-0.1030,0.1993,0.1025
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$172.68B,-0.1294,0.2103,0.1565
...,...,...,...,...,...,...,...,...
SDP,ProShares UltraShort Utilities,Inverse Equity: U.S. Utilities,ProShares,0.95%,$1.96M,0.0526,-0.2945,-0.2642
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$1.24M,-0.0942,0.1026,-0.0212
ERUS,iShares MSCI Russia ETF,Equity: Russia - Total Market,Blackrock,0.08%,$961.13K,0.0000,-0.3319,-0.2274


# Fazer os rankings de rentabilidade por período e somá-los

In [49]:
base_final['rank_1_anos'] = base_final['1 Year'].rank(ascending = False)
base_final['rank_3_anos'] = base_final['3 Years'].rank(ascending = False)
base_final['rank_5_anos'] = base_final['5 Years'].rank(ascending = False)
base_final['rank_final'] = (base_final['rank_1_anos']+
                                  base_final['rank_3_anos']+
                                  base_final['rank_5_anos'])

base_final

C:\Users\lilia\AppData\Local\Temp\ipykernel_15928\4013591438.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_final['rank_1_anos'] = base_final['1 Year'].rank(ascending = False)
C:\Users\lilia\AppData\Local\Temp\ipykernel_15928\4013591438.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_final['rank_3_anos'] = base_final['3 Years'].rank(ascending = False)
C:\Users\lilia\AppData\Local\Temp\ipykernel_15928\4013591438.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

,Name,Segment,Issuer,Expense Ratio,AUM,1 Year,3 Years,5 Years,rank_1_anos,rank_3_anos,rank_5_anos,rank_final
Ticker,,,,,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$374.42B,-0.0901,0.1991,0.1105,920.5,434.0,101.0,1455.5
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$306.66B,-0.0902,0.1997,0.1106,923.5,423.5,99.5,1446.5
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$287.12B,-0.0902,0.1998,0.1107,923.5,421.5,97.5,1442.5
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$282.79B,-0.1030,0.1993,0.1025,1022.0,431.5,151.5,1605.0
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$172.68B,-0.1294,0.2103,0.1565,1170.5,340.0,14.0,1524.5
...,...,...,...,...,...,...,...,...,...,...,...,...
SDP,ProShares UltraShort Utilities,Inverse Equity: U.S. Utilities,ProShares,0.95%,$1.96M,0.0526,-0.2945,-0.2642,100.0,1444.0,1456.0,3000.0
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$1.24M,-0.0942,0.1026,-0.0212,950.0,977.0,1299.0,3226.0
ERUS,iShares MSCI Russia ETF,Equity: Russia - Total Market,Blackrock,0.08%,$961.13K,0.0000,-0.3319,-0.2274,256.5,1449.0,1446.0,3151.5


## Ordenar do menor raking pro maior, encontrando os melhores ETFs.

In [52]:
base_final = base_final.sort_values(by= 'rank_final')

base_final.head(20)

,Name,Segment,Issuer,Expense Ratio,AUM,1 Year,3 Years,5 Years,rank_1_anos,rank_3_anos,rank_5_anos,rank_final
Ticker,,,,,,,,,,,,
UGA,United States Gasoline Fund LP,Commodities: Energy Gasoline,Marygold,0.93%,$74.99M,0.0931,0.7312,0.1455,59.0,2.0,20.0,81.0
ITB,iShares U.S. Home Construction ETF,Equity: U.S. Housing,Blackrock,0.39%,$1.64B,0.1548,0.3995,0.1200,20.0,37.0,50.0,107.0
AIRR,First Trust RBA American Industrial Renaissanc...,Equity: U.S. Industrials,First Trust,0.70%,$316.07M,0.0936,0.3608,0.1244,58.0,59.0,42.0,159.0
XLE,Energy Select Sector SPDR Fund,Equity: U.S. Energy,State Street Global Advisors,0.10%,$40.36B,0.1489,0.4855,0.1016,22.0,20.0,156.5,198.5
SGG,iPath Series B Bloomberg Sugar Subindex Total ...,Commodities: Agriculture Sugar,Barclays Capital Inc.,0.45%,$41.42M,0.2770,0.3192,0.1078,7.0,82.0,124.0,213.0
GRID,First Trust NASDAQ Clean Edge Smart Grid Infra...,Equity: Global Infrastructure,First Trust,0.58%,$719.10M,0.0181,0.3512,0.1558,156.0,65.0,16.0,237.0
IEO,iShares U.S. Oil & Gas Exploration & Productio...,Equity: U.S. Oil & Gas Exploration & Production,Blackrock,0.39%,$764.20M,0.0667,0.6024,0.1005,78.0,6.0,164.5,248.5
FENY,Fidelity MSCI Energy Index ETF,Equity: U.S. Energy,Fidelity,0.08%,$1.59B,0.1340,0.5000,0.0913,26.0,19.0,235.0,280.0
VDE,Vanguard Energy ETF,Equity: U.S. Energy,Vanguard,0.10%,$8.03B,0.1304,0.5050,0.0916,29.0,18.0,233.0,280.0
